In [1]:
import os
import pprint
import math
import time
from sklearn.model_selection import train_test_split
import sys
sys.path.append('../tfmodels')
import bottleneck as bn

import random
import numpy as np
from past.builtins import xrange
import pandas as pd

#from data import read_data
from memory_model import *

In [2]:
flags = tf.app.flags

flags.DEFINE_integer("nb_words", 20001, "term number in input sequence(zero mask)")
flags.DEFINE_integer("mem_size", 10, "the memory length of input sequence")
flags.DEFINE_integer("nhop", 6, "the number of hop layers")
flags.DEFINE_integer("embedding_size", 150, "word embedding size")
flags.DEFINE_float("init_hid", 0.1, "init_hid for q")
flags.DEFINE_float("init_std", 0.05, "init_std for training variable")

flags.DEFINE_float("linear_ratio", 0.5, "keep probability of drop out [0.9]")
flags.DEFINE_float("learning_rate", 0.01, "learning rate [0.001]")
flags.DEFINE_integer("batch_size", 1024, "batch size to use during training [128]")
flags.DEFINE_float("clip_gradients", 50.0, "clip gradients to this norm")
flags.DEFINE_integer("n_epochs", 100, "number of epoch to use during training")
flags.DEFINE_boolean("epoch_save", True, "save checkpoint or not in each epoch")
flags.DEFINE_integer("print_step", 500, "print step duraing training [100]")
flags.DEFINE_string("logs_dir", "logs/", "logs directory [logs/]")
flags.DEFINE_string("model_dir", "model/", "model directory [model/]")
flags.DEFINE_boolean("dir_clear", False, "clear the log and model directory")
flags.DEFINE_boolean("lr_annealing", True, "use lr annealing or not after each epoch [False]")
#flags.DEFINE_string("current_task_name", '_season_prediction', "current task name [self_prediction]")
flags.DEFINE_integer("gpu_id", 0, "default gpu id [0]")
flags.DEFINE_integer("gpu_num", 4, "gpu_num")

FLAGS = flags.FLAGS

In [3]:
h5_file_path = 'Data/url_first_24.h5'
h5f = h5py.File(h5_file_path, 'r')
urls = np.array(h5f['urls'])
#clicks = h5f['clicks']
#seasons = h5f['seasons']
seq_len = np.array(h5f['seq_len'])
#seq_len = np.array(seq_len)
#user_id = h5f['user_id']
id_map_path = 'Data/seq_url_map_24.csv'
id_map_df = pd.read_csv(id_map_path)
id_dict = dict(zip(id_map_df.id, id_map_df.page_url))

In [4]:
maxlen = 10
step = 1
samples = 0
for i in xrange(len(seq_len)):
    samples += len(range(0, seq_len[i] - maxlen, step))
    
previous_urls = np.empty(shape=(samples, maxlen))
next_urls = np.empty(shape=(samples))
global_step = 0
for i in xrange(len(urls)):
    url = urls[i][0:seq_len[i]]
    for j in range(0, seq_len[i] - maxlen, step):
        previous_urls[global_step] = url[j: j + maxlen]
        next_urls[global_step] = url[j + maxlen]
        global_step += 1

In [5]:
idxs = np.arange(0, samples)
#idxs = idxs[0:2000]
train_idxs, test_idxs = train_test_split(idxs, test_size=10240, random_state=52)
test_idxs = np.sort(test_idxs)

In [ ]:
graph_to_use = tf.Graph()
config = tf.ConfigProto(allow_soft_placement=True)
config.gpu_options.allow_growth=True
with tf.Session(graph=graph_to_use, config=config) as session:
    mem_model = MemN2NModel(FLAGS, session, 'memn2n_url_prediction')
    mem_model.build_model(type='self', accK=5, export_attention=True)
    mem_model.build_model_summary()
    display(mem_model.model_summary())
    mem_model.run([previous_urls,next_urls], train_idxs, test_idxs)

Initializing


,variable_name,variable_shape,parameters
0,global/Variable:0,[],1.0
1,gpu/memory/Ain_c/embedding_table:0,"[20001, 150]",3000150.0
2,gpu/memory/Ain_t/W:0,"[10, 150]",1500.0
3,gpu/memory/Cin_c/embedding_table:0,"[20001, 150]",3000150.0
4,gpu/memory/Cin_t/W:0,"[10, 150]",1500.0
5,gpu/momory_hops/hops_h/W:0,"[150, 150]",22500.0
6,gpu/prediction/fc/W:0,"[150, 20001]",3000150.0


('Epoch', 1, '... training ...')
('Minibatch', 500, '/', 'loss:', 4.2602043)
('Minibatch', 500, '/', 'accuracy:', 0.51367188)


In [6]:
graph_to_use = tf.Graph()
config = tf.ConfigProto(allow_soft_placement=True)
config.gpu_options.allow_growth=True
with tf.Session(graph=graph_to_use, config=config) as session:
    mem_model = MemN2NModel(FLAGS, session, 'memn2n_url_prediction')
    mem_model.build_model(type='self', accK=5, export_attention=True)
    mem_model.build_model_summary()
    display(mem_model.model_summary())
    mem_model.model_restore()
    _, testMetric, results = mem_model.model_run([previous_urls,next_urls], 
                                                  test_idxs, run_type='self',
                                                  mode='test', shuffle=False, save_metric=True)

Initializing


,variable_name,variable_shape,parameters
0,global/Variable:0,[],1.0
1,gpu/memory/Ain_c/embedding_table:0,"[20001, 150]",3000150.0
2,gpu/memory/Ain_t/W:0,"[10, 150]",1500.0
3,gpu/memory/Cin_c/embedding_table:0,"[20001, 150]",3000150.0
4,gpu/memory/Cin_t/W:0,"[10, 150]",1500.0
5,gpu/momory_hops/hops_h/W:0,"[150, 150]",22500.0
6,gpu/prediction/fc/W:0,"[150, 20001]",3000150.0


Load Model ...


In [8]:
prediction = np.concatenate([np.argsort(-results[i][0])[:,:5] for i in range(0, len(results))])
#prediction = np.concatenate([bn.argpartition(-results[0][0], 5)[:,:5] for i in range(0, len(results))])
attention = np.concatenate([results[i][1] for i in range(0, len(results))]) 

In [42]:
i = np.random.randint(0,10240)
temp_idx = test_idxs[i]
if next_urls[temp_idx] in prediction[i]:
    print('\n'.join(['{}: {}'.format(id_dict[id], attention[i][0][idx]) for idx, id 
           in enumerate(previous_urls[temp_idx])]))
    print('\nprediction:')
    print('\n'.join([id_dict[id] for id in prediction[i]]))
    print('\ngroundtruth:')
    print(id_dict[next_urls[temp_idx]])

hulu.com/tv: 0.113912783563
$: 0.00927216187119
hulu.com/: 0.0386288017035
hulu.com/catfish-the-tv-show: 0.0580177605152
hulu.com/watch/433554 (Catfish: The TV Show) (episode): 0.0368508473039
hulu.com/watch/434899 (Catfish: The TV Show) (episode): 0.0706789940596
hulu.com/watch/439797 (Catfish: The TV Show) (episode): 0.0862120464444
hulu.com/watch/439798 (Catfish: The TV Show) (episode): 0.157057926059
$: 0.0237574372441
hulu.com/: 0.405611276627

prediction:
hulu.com/catfish-the-tv-show
hulu.com/search?q=catfish%3a the tv show
$
hulu.com/tv
hulu.com/

groundtruth:
hulu.com/catfish-the-tv-show
